In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./datasetV2.csv')

In [8]:
df.iloc[0]

RANK                                              1
NAME                                       kaicenat
LANGUAGE                                    English
TYPE                                    personality
MOST_STREAMED_GAME                    Just Chatting
2ND_MOST_STREAMED_GAME            I'm Only Sleeping
AVERAGE_STREAM_DURATION                         7.6
FOLLOWERS_GAINED_PER_STREAM                   18405
AVG_VIEWERS_PER_STREAM                        15852
AVG_GAMES_PER_STREAM                            2.3
TOTAL_TIME_STREAMED                            4698
TOTAL_FOLLOWERS                            10600000
TOTAL_VIEWS                                 9150000
TOTAL_GAMES_STREAMED                            194
ACTIVE_DAYS_PER_WEEK                            3.6
MOST_ACTIVE_DAY                              Friday
DAY_WITH_MOST_FOLLOWERS_GAINED             Saturday
Name: 0, dtype: object